In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [2]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block

In [3]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    #solver = HistGradientBoostingRegressor() #TRYING DIFFERNT SOLVER TO ENSURE CONVERGENCE



    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'    
    default_output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': None,
        'tau': None,
        'theta': None,
        'E_tau_theta_results': None,
        'ccm_value': None,
        'convergence_p_value': None,
        'correlation': None
    }
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        try:
            c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        except:
            print(f"{interaction} did not converge")
            return default_output
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    try:
        convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)
    except:
        print(f"{interaction} did not converge")
        return default_output

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [4]:
HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
HAB_data = HAB_data.set_index('time')
target = 'Avg_Chloro'

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data

,AVG_DENS_kgm3,AVG_SAL_PSU,AVG_TEMP_C,Avg_Chloro,BOT_DENS_kgm3,BOT_SAL_PSU,BOT_TEMP_C,Nitrate,Nitrite,Phosphate,SURF_DENS_kgm3,SURF_SAL_PSU,SURF_TEMP_C,Silicate,WSPD
time,,,,,,,,,,,,,,,
0,1024.129374,33.695,18.10,1.82,1024.620655,33.62,15.9,0.43,0.03,0.24,1023.638094,33.77,20.3,5.6,4.691667
7,1024.289049,33.640,17.35,2.71,1024.391719,33.62,16.9,0.83,0.04,0.37,1024.186379,33.66,17.8,5.1,2.787500
13,1023.563298,33.660,20.25,0.99,1024.026040,33.58,18.3,0.45,0.00,0.24,1023.100557,33.74,22.2,3.5,3.412500
20,1023.740812,33.650,9.90,1.23,1023.780535,33.62,19.4,0.34,0.00,0.17,1023.701088,33.68,19.8,4.2,3.495833
28,1023.004896,33.710,11.25,2.41,1023.000725,33.69,22.5,0.29,0.00,0.12,1023.009067,33.73,22.5,3.8,2.604167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,1024.149572,33.470,17.40,3.22,1024.152950,33.46,17.4,0.85,0.00,0.30,1024.146195,33.48,17.4,6.2,5.076389
5572,1024.033476,33.475,17.90,3.39,1024.040616,33.47,17.9,0.45,0.00,0.25,1024.026335,33.48,17.9,4.9,2.737500
5579,1024.214172,33.545,17.35,7.82,1024.454252,33.55,16.4,0.00,0.00,0.17,1023.974093,33.54,18.3,5.2,3.306250


In [5]:
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
HAB_data = HAB_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(HAB_data)
HAB_data = Imputer.transform(HAB_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
HAB_data

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_45532/137097351.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_45532/137097351.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


,AVG_DENS_kgm3,AVG_SAL_PSU,AVG_TEMP_C,Avg_Chloro,BOT_DENS_kgm3,BOT_SAL_PSU,BOT_TEMP_C,Nitrate,Nitrite,Phosphate,SURF_DENS_kgm3,SURF_SAL_PSU,SURF_TEMP_C,Silicate,WSPD
time,,,,,,,,,,,,,,,
0,1024.129374,33.695,18.10,1.82,1024.620655,33.62,15.9,0.43,0.03,0.24,1023.638094,33.77,20.3,5.6,4.691667
7,1024.289049,33.640,17.35,2.71,1024.391719,33.62,16.9,0.83,0.04,0.37,1024.186379,33.66,17.8,5.1,2.787500
13,1023.563298,33.660,20.25,0.99,1024.026040,33.58,18.3,0.45,0.00,0.24,1023.100557,33.74,22.2,3.5,3.412500
20,1023.740812,33.650,9.90,1.23,1023.780535,33.62,19.4,0.34,0.00,0.17,1023.701088,33.68,19.8,4.2,3.495833
28,1023.004896,33.710,11.25,2.41,1023.000725,33.69,22.5,0.29,0.00,0.12,1023.009067,33.73,22.5,3.8,2.604167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,1024.149572,33.470,17.40,3.22,1024.152950,33.46,17.4,0.85,0.00,0.30,1024.146195,33.48,17.4,6.2,5.076389
5572,1024.033476,33.475,17.90,3.39,1024.040616,33.47,17.9,0.45,0.00,0.25,1024.026335,33.48,17.9,4.9,2.737500
5579,1024.214172,33.545,17.35,7.82,1024.454252,33.55,16.4,0.00,0.00,0.17,1023.974093,33.54,18.3,5.2,3.306250


In [6]:
print(HAB_data.isna().sum())

AVG_DENS_kgm3     0
AVG_SAL_PSU       0
AVG_TEMP_C        0
Avg_Chloro        0
BOT_DENS_kgm3     0
BOT_SAL_PSU       0
BOT_TEMP_C        0
Nitrate           0
Nitrite           0
Phosphate         0
SURF_DENS_kgm3    0
SURF_SAL_PSU      0
SURF_TEMP_C       0
Silicate          0
WSPD              0
dtype: int64


In [7]:
#STANDARDIZE
#LEFT OFF HERE THERE IS AN ERROR
'''
scaler = MinMaxScaler()
columns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', 
           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', 
           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', 
           'Silicate', 'WSPD']
HAB_data[columns] = scaler.fit_transform(HAB_data[columns])
HAB_data
'''

"\nscaler = MinMaxScaler()\ncolumns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', \n           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', \n           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', \n           'Silicate', 'WSPD']\nHAB_data[columns] = scaler.fit_transform(HAB_data[columns])\nHAB_data\n"

In [8]:
block = get_block(HAB_data, num_lags=50, tau=1)
block

,AVG_DENS_kgm3(t-0),AVG_SAL_PSU(t-0),AVG_TEMP_C(t-0),Avg_Chloro(t-0),BOT_DENS_kgm3(t-0),BOT_SAL_PSU(t-0),BOT_TEMP_C(t-0),Nitrate(t-0),Nitrite(t-0),Phosphate(t-0),...,BOT_SAL_PSU(t+50),BOT_TEMP_C(t+50),Nitrate(t+50),Nitrite(t+50),Phosphate(t+50),SURF_DENS_kgm3(t+50),SURF_SAL_PSU(t+50),SURF_TEMP_C(t+50),Silicate(t+50),WSPD(t+50)
time,,,,,,,,,,,,,,,,,,,,,
0,1024.129374,33.695,18.10,1.82,1024.620655,33.62,15.9,0.43,0.03,0.24,...,33.66,19.3,0.37,0.0,0.28,1023.818956,33.70,19.4,6.3,4.591667
7,1024.289049,33.640,17.35,2.71,1024.391719,33.62,16.9,0.83,0.04,0.37,...,33.67,20.0,0.34,0.0,0.24,1023.686670,33.73,20.0,5.4,3.308333
13,1023.563298,33.660,20.25,0.99,1024.026040,33.58,18.3,0.45,0.00,0.24,...,33.66,20.6,0.32,0.0,0.28,1023.525345,33.69,20.5,4.2,3.883333
20,1023.740812,33.650,9.90,1.23,1023.780535,33.62,19.4,0.34,0.00,0.17,...,33.52,13.6,0.71,0.0,0.32,1023.998606,33.54,18.2,4.7,2.491667
28,1023.004896,33.710,11.25,2.41,1023.000725,33.69,22.5,0.29,0.00,0.12,...,33.48,17.2,0.17,0.0,0.26,1023.662548,33.56,19.6,0.0,2.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,1024.149572,33.470,17.40,3.22,1024.152950,33.46,17.4,0.85,0.00,0.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5572,1024.033476,33.475,17.90,3.39,1024.040616,33.47,17.9,0.45,0.00,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5579,1024.214172,33.545,17.35,7.82,1024.454252,33.55,16.4,0.00,0.00,0.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
E_list = range(2,13)
tau_list = [-1,-2,-3] #can try more taus
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=4)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

There are 28 interactions
('AVG_SAL_PSU', 'Avg_Chloro')
('AVG_DENS_kgm3', 'Avg_Chloro')
('AVG_TEMP_C', 'Avg_Chloro')
('Avg_Chloro', 'AVG_DENS_kgm3')
('Avg_Chloro', 'AVG_SAL_PSU')
('Avg_Chloro', 'AVG_TEMP_C')
('Avg_Chloro', 'BOT_DENS_kgm3')
('Avg_Chloro', 'BOT_SAL_PSU')
('Avg_Chloro', 'BOT_TEMP_C')
('Avg_Chloro', 'Nitrate')
('Avg_Chloro', 'Nitrite')
('Avg_Chloro', 'Phosphate')


/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:119: RuntimeWarning: divide by zero encountered in divide
  distRowScale = self.theta / distRowMean
/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:120: RuntimeWarning: invalid value encountered in multiply
  W = exp( -distRowScale[:,None] * self.knn_distances )


 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Avg_Chloro', 'Nitrite') did not converge
('Avg_Chloro', 'SURF_DENS_kgm3')
('Avg_Chloro', 'SURF_SAL_PSU')
('Avg_Chloro', 'SURF_TEMP_C')
('Avg_Chloro', 'Silicate')
('Avg_Chloro', 'WSPD')
('BOT_DENS_kgm3', 'Avg_Chloro')
('BOT_SAL_PSU', 'Avg_Chloro')
('BOT_TEMP_C', 'Avg_Chloro')
('Nitrate', 'Avg_Chloro')
('Nitrite', 'Avg_Chloro')
('Phosphate', 'Avg_Chloro')
('SURF_DENS_kgm3', 'Avg_Chloro')
('SURF_SAL_PSU', 'Avg_Chloro')
('SURF_TEMP_C', 'Avg_Chloro')
('Silicate', 'Avg_Chloro')
('WSPD', 'Avg_Chloro')


In [19]:
# Get CCM results that show convergence (convergence p-value < 0.05)

ccm_cutoff = -1

significant_results = results_df[results_df.convergence_p_value<0.05]
significant_results = significant_results.sort_values(by='ccm_value', ascending=False)
significant_results = significant_results[['target (driver)', 'lib (driven)', 'E', 'tau', 'theta', 'ccm_value']].reset_index(drop=True)

display(significant_results[significant_results.ccm_value>ccm_cutoff])

# Choose system variables where the CCM value to or from the target is > ccm_cutoff
system_variables = significant_results[significant_results.ccm_value > ccm_cutoff]
system_variables = system_variables[['target (driver)', 'lib (driven)']].values.flatten().tolist()
system_variables = list(set(system_variables))
print('system variables: ')
display(sorted(system_variables))

,target (driver),lib (driven),E,tau,theta,ccm_value
0,Avg_Chloro,AVG_SAL_PSU,12.0,-3.0,5.0,0.520578
1,Avg_Chloro,SURF_TEMP_C,11.0,-3.0,7.0,0.368707
2,Nitrate,Avg_Chloro,10.0,-2.0,3.0,0.210130


system variables: 


['AVG_SAL_PSU', 'Avg_Chloro', 'Nitrate', 'SURF_TEMP_C']

In [18]:
results_df

,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
0,AVG_DENS_kgm3,Avg_Chloro,12.0,-3.0,5.0,E tau theta rho 0 2.0 -1.0...,0.464755,0.287718,0.101784
1,AVG_SAL_PSU,Avg_Chloro,12.0,-3.0,2.0,E tau theta rho 0 2.0 -1.0...,0.300224,1.000000,0.016363
2,AVG_TEMP_C,Avg_Chloro,12.0,-3.0,3.0,E tau theta rho 0 2.0 -1.0...,0.208638,0.164677,0.035169
3,Avg_Chloro,AVG_DENS_kgm3,12.0,-3.0,5.0,E tau theta rho 0 2.0 -1.0...,0.327618,0.926511,0.101784
4,Avg_Chloro,AVG_SAL_PSU,12.0,-3.0,5.0,E tau theta rho 0 2.0 -1.0...,0.520578,0.005564,0.016363
5,Avg_Chloro,AVG_TEMP_C,12.0,-2.0,9.0,E tau theta rho 0 2.0 -1.0...,0.252024,0.630805,0.035169
6,Avg_Chloro,BOT_DENS_kgm3,12.0,-3.0,4.0,E tau theta rho 0 2.0 -1.0...,0.220885,0.936445,0.109979
7,Avg_Chloro,BOT_SAL_PSU,12.0,-3.0,4.0,E tau theta rho 0 2.0 -1.0...,0.469439,0.182658,0.030527
8,Avg_Chloro,BOT_TEMP_C,12.0,-3.0,4.0,E tau theta rho 0 2.0 -1.0...,0.227550,0.983508,-0.100140
9,Avg_Chloro,Nitrate,2.0,-3.0,9.0,E tau theta rho 0 2.0 -1.0...,0.185546,0.573059,-0.028910
